In [ ]:
import arablepy
client = arablepy.ArableClient()
from   sensor_includes   import email, password_ccber
client.connect(email = email, password = password_ccber)

from   datetime          import timedelta
from   datetime          import datetime
from   io                import StringIO
import os

import re
import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates
import altair            as alt

# register_matplotlib_converters()
# Enable altair usage in Jupyter Notebook
alt.renderers.enable('html')

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters

pd.options.display.max_columns = None
pd.set_option('max_colwidth', 40)

In [ ]:
data_dictionary = dict()
available_tables = client.schema(df = False)
available_tables

# Make a dictionary of original column names (keys) and descriptive column names (values)
# description, data_type, column_name
for table in available_tables:
    try:
        table_data_dictionary = client.schema(table, df=True)
        table_data_dictionary_columns = table_data_dictionary["column_name"]
        table_data_dictionary = table_data_dictionary.drop("column_name", axis="columns")
        table_data_dictionary = table_data_dictionary.drop("data_type", axis="columns")
        table_data_dictionary = table_data_dictionary.set_index(table_data_dictionary_columns)
        table_data_dictionary = table_data_dictionary["description"].to_dict()
        data_dictionary[table] = table_data_dictionary
    except: # HTTPError - 500 Server Error: Internal Server Error for url
        continue


devices = client.devices(df=True)
device_locations = devices["current_location"]
test = dict()
for key in list(device_locations[0].keys()):
    try:
        device_location_names = [i[key] for i in device_locations]
        test[key] = device_location_names
    except KeyError:
        continue
device_data = pd.DataFrame.from_dict(test)

device_data = device_data[device_data["addr_state"] == "CA"]

def find_dates(index: int, option: str):
    start = datetime.strptime(index, "%Y-%m-%dT%H:%M:%S")
    end = datetime.now()
    dates = []

    for i in range(1, 12):
        for j in [start.year, end.year]:
            date = start.replace(year = j).replace(month = i + 1).replace(day = 1
                   ).replace(hour = 0).replace(minute = 0).replace(second = 0)
            if start <= date <= datetime.now():
                dates.append(date)
                
    if option == "start":
        dates.insert(0, start)
    elif option == "end":
        dates.insert(-1, end)
        
    dates = sorted([date.strftime("%Y-%m-%dT%H:%M:%S") for date in dates])
    dates = [f"{date}Z" for date in dates]
    return dates

In [ ]:
def VWC(df):
    raw_VWC = df['sdi12_value_0'].to_list()
    calc_VWC = []
    for raw in raw_VWC:
        raw = float(raw)
        calc = 4.3000 * 10**(-6) * raw**(3) 
        - 5.50 * 10**(-4) * raw**(2) 
        + 2.92 * 10**(-2) * raw - 5.30 * 10**(-2)
        calc_VWC.append(calc)
    return calc_VWC

In [ ]:
def time_and_date(df):
    tm = df['time'].to_list()
    date_stamp = []
    time_stamp = []
    for i in range(len(tm)):
        date_stamp.append(str(tm[i])[:10])
        time_stamp.append(str(tm[i]))
    return date_stamp, time_stamp

start_dates = []
end_dates = []
for i in list(device_data["start_date"]):
    try:
        start_dates.append(find_dates_years(i[:19], "start"))
        end_dates.append(find_dates_years(i[:19], "end"))
    except:
        continue

In [ ]:
def retrieve_dataframe(data_type, device_id, start_time, end_time):
    """
    cursor (string)          : Encoded cursor token (for pagination, from X-Cursor-Next response header)
    limit (integer)          : [ 1...10000 ]; Default: 1000
    order (string)           : Default: "asc". Enum: "asc" "desc"
    temp (string)            : Enum: "C" "F". Temperature unit in either [C]elsius or [F]ahrenheit
    pres (string)            : Enum: "mb" "kp" Pressure unit in either millibars [mb] or kilopascals [kp]
    ratio (string)           : Enum: "dec" "pct" Ratio either as percent [pct] or decimal value [dec]
    size (string)            : Enum: "in" "mm" Size unit in either [in]ches or millimeters [mm]
    speed (string)           : Enum: "mps" "kph" "mph" Speed unit; meters per second [mps], 
                               kilometers per hour [kph], or miles per hour [mph]
    device (string)          : Device name, e.g., A000123 (required if location not specified)
    location (string)        : Location ID (required if device not specified)
    local_time (string)      : Local time column specified as timezone name, offset seconds or 
                               ISO format(e.g. America/Los_Angeles, -14400, -10: 30)(optional)
    select (Array of strings): Comma-separated column list, e.g., time,device,location,tair
    start_time (string)      : Start date/time, e.g., 2019-01-01 or 2019-01-01T00:00:00Z
    end_time (string)        : End date/time, e.g., 2019-01-01 or 2019-01-01T00:00:00Z
    """
        
    df = client.data(data_type, 
                     devices = [device_id], 
                     start_time = start_time,
                     end_time = end_time)
    
    df = df.dropna(axis = 'columns', thresh=2)  # how='all')
    
    try:
        df = df.rename(mapper = data_dictionary[data_type], axis='columns') 
    except:
        pass
        
    columns_to_remove    = {"daily": ['location','time','lat','long'],
                            "hourly": ['location','time','lat','long'],
                            "aux_raw": ['location','sdi12_version','sdi12_vendor_id','sdi12_sensor_model','sdi12_sensor_version','sdi12_sensor_sn'],
                            "health": ['location','time','lat','long'],
                            "local_hourly": ['location','time','lat','long'], 
                            "location_irrigation_forecast_daily": ['location','time','lat','long'], 
                            "sentek_daily": ['location','time','lat','long'], 
                            "sentek_hourly": ['location','time','lat','long']}
    
    for i in columns_to_remove[data_type]:
        try:                
            df = df.drop(i, axis='columns')
        except:
            pass
            
    if data_type == "aux_raw":
        calc_VWC = VWC(df)
        df.insert(6, '5TE Calibrated VWC (m^3/m^3)', calc_VWC)
    
    return df

In [ ]:
device_data = device_data[device_data["device_name"] == "A000563"]["name"]

In [ ]:
device_data["device_name"]

In [ ]:
daily_data = retrieve_dataframe("daily", "A000563", '2020-06-01', '2020-07-01')
hourly_data = retrieve_dataframe("hourly", "A000563", '2020-06-01', '2020-07-01')

In [ ]:
#hourly_data

In [ ]:
def altair_plot(df, color, dimensions, q, q_name, time):                
    df[time] = pd.to_datetime(df[time], infer_datetime_format=True)
    device_id = device_data.to_list()[0]
    start_date = str(df[time][0]).split()[0]
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #df = df.fillna(0)
    #df = df.replace(np.inf, 0)
    df = df.loc[:, q:time] 
    #print(df)
    end_date = str(df[time][len(df[time]) - 1]).split()[0]
    base = alt.Chart(df,
                     title = f"{device_id} ({start_date} - {end_date})"
                    ).mark_line(color = color
                    ).properties(width  = dimensions[0], 
                                 height = dimensions[1]
                    ).encode(x = alt.X(f'monthdate({time}):O', axis=alt.Axis(title='')),
                             y = alt.Y(f"{q}:Q", axis=alt.Axis(title=q_name)))
    return base
                    

In [ ]:
max_temp_hourly = altair_plot(hourly_data, "cyan", [300, 300], "The maximum air temperature experienced during this hour", "Max.", "UTC time")
min_temp_hourly = altair_plot(hourly_data, "magenta", [300, 300], "The minimum air temperature experienced during this hour", "Min.", "UTC time")
graph_hourly = alt.layer(max_temp_hourly, min_temp_hourly)
graph_hourly

In [ ]:
max_temp_daily = altair_plot(daily_data, "black", [300, 300], "Daily maximum temperature (C)", "Max.", 'Local date')
#mean_leaf_ground_temp_daily = altair_plot(daily_data, "blue", [800, 200], "Daily mean leaf/ground temperature (C)", "Mean. leaf/ground", 'Local date')
mean_temp_daily = altair_plot(daily_data, "blue", [300, 300], "Daily mean temperature (C)", "Mean.", 'Local date')
#dew_temp_daily = altair_plot(daily_data, "magenta", [800, 200], "Dew temperature (C)", "Dew", 'Local date')
min_temp_daily = altair_plot(daily_data, "rebeccapurple", [300, 300], "Daily minimum temperature (C)", "Min.", 'Local date')
graph_daily = alt.layer(max_temp_daily, mean_temp_daily, min_temp_daily, max_temp_hourly, min_temp_hourly)
graph_daily